In [ ]:
import numpy as np
import matplotlib.pyplot as plt

l = 0.2 #length of container
#function to generate radial thermal resistance
def Rth_radial(r1,r2, k):
    Rth = np.log(r2/r1) / (2*np.pi*l*k)
    return Rth

In [3]:
#Approach: generate thermal resistances of key insulation components and run basic simulation

#Defining key variables for cooler design. This model IGNORES plastic conduction.
#put all of this into a function to output performance for a variety of designs?
R_ice = 0.08 
#first insulation layer between ice and vaccines
t_insulation = 0.005 #stops vaccine freezing
R_insulation = R_ice + t_insulation 
k_insulation = 0.03
Rth_insulation = Rth_radial(R_ice, R_insulation, k_insulation)

# air gap modeled with natural convection instead of conduction
t_air = 0.04
R_air = R_insulation + t_air

# estimate natural convection coefficient (vertical annular gap / conservative)
h_air = 3.0  # W/m²K (can range from 2 to 10 for natural convection in air)

A_air = 2 * np.pi * R_insulation * l  # cylindrical side area at R_insulation
Rth_air = 1 / (h_air * A_air)




#foam outer layer 
t_foam = 0.08
R_foam = R_air + t_foam
k_foam = 0.03
Rth_foam = Rth_radial(R_air, R_foam, k_foam)
#printing total part diameter, for refernce:
print(str(R_foam*2*100) + "cm") 

41.0cm


In [4]:
#Convection loss estimation
v = 1 #this is speed of external air
h_outer = 12.12-1.16*v+11.6*v**(1/2) # #empirical relationship https://www.engineeringtoolbox.com/convective-heat-transfer-d_430.html - this is an empirical equation and can be used for velocities 2 to 20 m/s.
Rth_convection = 1/(2*np.pi*R_foam*l*h_outer)

#total thermal resistance 
Rth_total =  Rth_foam + Rth_air + Rth_insulation + Rth_convection

#print(Rth_convection)

#print(Rth_total)

In [7]:
#Flat surface simulation
#Constructed of insulated and foam layer
def Rth_linear(l, A, k):
    Rth = l/(A*k)
    return Rth

A = np.pi * R_ice * R_ice

#first insulation layer, the same layer used around ice bottle circumfrence
t_flat_insulation = t_insulation

Rth_Flat_Insulation = Rth_linear(t_flat_insulation, A, k_insulation)

#second insulation layer 
t_flat_foam = 0.05

Rth_flat_foam = Rth_linear(t_flat_foam, A , k_insulation)

Rth_flat_total = Rth_Flat_Insulation + Rth_flat_foam

In [6]:
#setting up simulation
T_e = 30 #degc, environment temp
T_ice = 0 #degc, ice temp
del_T = T_e - T_ice
Q_dot_radial = del_T / Rth_total #using ohms analogy
Q_dot_linear = 2 * del_T / Rth_flat_total
Q_dot = Q_dot_linear + Q_dot_radial
print("rate of heat loss: " +str(round(Q_dot,2)) +" W")

rate of heat loss: 2.32 W


In [8]:
#Estimating phase change duration
m_ice = 2.5 #kg
SLH_ice = 334000 #j/kg
Q_ice = SLH_ice * m_ice

t_cool = Q_ice/Q_dot # seconds
t_cool_hours = t_cool / 3600

print("Time at adequate temp: " + str(round(t_cool_hours,1)) + " hours")

Time at adequate temp: 99.9 hours


In [9]:
#Calculating temperature at each boundary 
del_T_insulation = Q_dot * Rth_insulation 
print("temperature at edge of insulation layer: " +str(round(del_T_insulation,1)) + " deg C")
del_T_air = Q_dot * Rth_air
T_air_edge = del_T_air + del_T_insulation
print("temperature at edge of vaccine container: " +str(round(T_air_edge,1)) + " deg C")

temperature at edge of insulation layer: 3.7 deg C
temperature at edge of vaccine container: 11.0 deg C
